In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import yfinance as yf
from scipy.stats import kurtosis, skew

class SPYDataAnalyzer:
    def __init__(self, years=10, cache_file='spy_data.csv'):
        self.years = years
        self.cache_file = cache_file
        self.data = None
    
    def fetch_data(self):
        # Check if data is cached and is up-to-date
        if os.path.exists(self.cache_file):
            modified_time = datetime.fromtimestamp(os.path.getmtime(self.cache_file))
            if datetime.now() - modified_time < timedelta(days=1):
                self.data = pd.read_csv(self.cache_file, parse_dates=['Date'], index_col='Date')
                return

        # Fetch data from Yahoo Finance
        end_date = datetime.now()
        start_date = end_date - timedelta(days=self.years * 365)
        spy = yf.download('SPY', start=start_date, end=end_date)
        self.data = spy[['Open', 'High', 'Low', 'Close']]
        self.data.to_csv(self.cache_file)
    
    def calculate_indicators(self):
        if self.data is None:
            raise ValueError("No data available. Please fetch data first.")
        
        # Calculate moving averages
        self.data['20_MA'] = self.data['Close'].rolling(window=20).mean()
        self.data['52_Week_MA'] = self.data['Close'].rolling(window=260).mean()
        
        # Calculate Bollinger Bands for 20-day and 52-week
        self.data['20_Upper_BB'] = self.data['20_MA'] + 2 * self.data['Close'].rolling(window=20).std()
        self.data['20_Lower_BB'] = self.data['20_MA'] - 2 * self.data['Close'].rolling(window=20).std()
        self.data['52_Upper_BB'] = self.data['52_Week_MA'] + 2 * self.data['Close'].rolling(window=260).std()
        self.data['52_Lower_BB'] = self.data['52_Week_MA'] - 2 * self.data['Close'].rolling(window=260).std()
        
        # Calculate daily returns and other statistics
        self.data['Daily_Return'] = self.data['Close'].pct_change()
        self.data['Cumulative_Return'] = (1 + self.data['Daily_Return']).cumprod() - 1
        self.data['Std_Dev'] = self.data['Daily_Return'].rolling(window=20).std()
        
        # Calculate statistical metrics only where there are enough data points
        self.data['Skewness'] = self.data['Daily_Return'].rolling(window=20).apply(lambda x: skew(x.dropna()), raw=False)
        self.data['Kurtosis'] = self.data['Daily_Return'].rolling(window=20).apply(lambda x: kurtosis(x.dropna()), raw=False)
        self.data['Positive_Return_Percentage'] = self.data['Daily_Return'].rolling(window=20).apply(lambda x: (x > 0).mean() * 100, raw=False)
        
        # Calculate Kelly ratio and fraction based on available returns
        win_rate = self.data['Positive_Return_Percentage'] / 100
        loss_rate = 1 - win_rate
        avg_win = self.data[self.data['Daily_Return'] > 0]['Daily_Return'].mean()
        avg_loss = abs(self.data[self.data['Daily_Return'] <= 0]['Daily_Return'].mean())
        self.data['Kelly_Ratio'] = win_rate - (loss_rate / (avg_win / avg_loss))
        self.data['Kelly_Fraction'] = self.data['Kelly_Ratio'] / 2
        
        # Drop initial rows with NaN values after calculations
        self.data.dropna(inplace=True)

    def get_data(self):
        return self.data
    
    def save_to_csv(self, filename='spy_analyzed_data.csv'):
        if self.data is not None:
            self.data.to_csv(filename)
        else:
            raise ValueError("No data available to save. Please fetch and process data first.")

# Example usage
analyzer = SPYDataAnalyzer(years=10)
analyzer.fetch_data()
analyzer.calculate_indicators()
data = analyzer.get_data()
print(data.head())

In [ ]:
import yfinance as yf
import yoptions as yo

class OptionsDataFetcher:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)
    
    def fetch_options_data(self, expiration_date, strikes):
        try:
            options_chain = self.stock.option_chain(expiration_date)
        except ValueError as e:
            print(f"Error fetching options data: {e}")
            available_expirations = self.stock.options
            print(f"Available expirations are: {available_expirations}")
            expiration_date = available_expirations[0]  # Use the first available expiration date
            options_chain = self.stock.option_chain(expiration_date)
        
        options_data = {}
        for strike in strikes:
            option_data = options_chain.puts[options_chain.puts['strike'] == strike]
            if not option_data.empty:
                options_data[strike] = option_data
            else:
                print(f"No data available for strike price {strike}")
        
        return options_data

    def display_options_data(self, options_data):
        for strike, data in options_data.items():
            print(f"Option at strike price {strike}:")
            print(data)
            print("\n")

class BullPutStrategy:
    def __init__(self, options_data, strikes):
        self.options_data = options_data
        self.strikes = strikes
    
    def calculate_net_premium(self):
        premiums = []
        for strike in self.strikes:
            if strike in self.options_data and not self.options_data[strike].empty:
                premiums.append(self.options_data[strike]['lastPrice'].values[0])
            else:
                print(f"No premium data available for strike price {strike}")
                premiums.append(0)
        
        if len(premiums) < 2:
            raise ValueError("Not enough valid premiums to calculate net premium.")
        
        net_premium = premiums[0] - sum(premiums[1:])
        return net_premium

class BullPutLadder(BullPutStrategy):
    def __init__(self, options_data, strikes):
        super().__init__(options_data, strikes)
import datetime

# Example usage
ticker = 'SPY'
today = datetime.date.today()
next_week = today + datetime.timedelta(days=7)
expiration_date = next_week.strftime('%Y-%m-%d')

# Fetch the current close price of SPY
spy = yf.Ticker(ticker)
current_close = spy.history(period='1d')['Close'].iloc[-1]

# Define the strike prices for the bull put ladder strategy closer to the current close
strikes = [round(current_close * 0.95), round(current_close * 0.90), round(current_close * 0.85)]

fetcher = OptionsDataFetcher(ticker)
options_data = fetcher.fetch_options_data(expiration_date, strikes)
fetcher.display_options_data(options_data)

bull_put_ladder = BullPutLadder(options_data, strikes)
try:
    net_premium = bull_put_ladder.calculate_net_premium()
    print(f"\nNet Premium Received/Paid for the Bull Put Ladder: {net_premium}")
except ValueError as e:
    print(f"Error calculating net premium: {e}")


In [ ]:
class BearCallSpread:
    def __init__(self, options_data, strikes):
        self.options_data = options_data
        self.strikes = strikes
    
    def calculate_net_premium(self):
        premiums = []
        for strike in self.strikes:
            if strike in self.options_data and not self.options_data[strike].empty:
                premiums.append(self.options_data[strike]['lastPrice'].values[0])
            else:
                print(f"No premium data available for strike price {strike}")
                premiums.append(0)
        
        if len(premiums) < 2:
            raise ValueError("Not enough valid premiums to calculate net premium.")
        
        net_premium = premiums[0] - sum(premiums[1:])
        return net_premium

class BearCallLadder(BearCallSpread):
    def __init__(self, options_data, strikes):
        super().__init__(options_data, strikes)

# Example usage for Bear Call Spread
ticker = 'SPY'
expiration_date = '2024-10-31'

# Define the strike prices for the bear call spread strategy
strikes = [555, 570]

fetcher = OptionsDataFetcher(ticker)
options_data = fetcher.fetch_options_data(expiration_date, strikes)
fetcher.display_options_data(options_data)

bear_call_spread = BearCallSpread(options_data, strikes)
try:
    net_premium = bear_call_spread.calculate_net_premium()
    print(f"\nNet Premium Received/Paid for the Bear Call Spread: {net_premium}")
except ValueError as e:
    print(f"Error calculating net premium: {e}")

# Example usage for Bear Call Ladder
# Define the strike prices for the bear call ladder strategy
strikes = [555, 570, 572]

fetcher = OptionsDataFetcher(ticker)
options_data = fetcher.fetch_options_data(expiration_date, strikes)
fetcher.display_options_data(options_data)

bear_call_ladder = BearCallLadder(options_data, strikes)
try:
    net_premium = bear_call_ladder.calculate_net_premium()
    print(f"\nNet Premium Received/Paid for the Bear Call Ladder: {net_premium}")
except ValueError as e:
    print(f"Error calculating net premium: {e}")
